In [0]:
""" 
author:-aam35

"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import tensorflow.contrib.eager as tfe
from sklearn.decomposition import PCA




tf.executing_eagerly()
# Define paramaters for the model
learning_rate = 0.001
batch_size = 200
n_epochs = 200
n_train = 55000
n_test = 10000

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
train, val, test = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/',one_hot=True,)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 26421880 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 29515 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 4422102 bytes.
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 5148 bytes.
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/data

In [0]:
X =np.append(train.images,test.images,axis=0)

In [0]:
pca = PCA(0.99)

In [22]:
pca.fit(X)


PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [0]:
img, label = train.next_batch(batch_size)

In [0]:
w = tf.Variable(tf.random_normal([pca.transform(train.images).shape[1],train.labels.shape[1]],stddev=0.01))
b = tf.Variable(tf.zeros([1 , train.labels.shape[1]],dtype=tf.float32))


In [0]:
def model(x):
  U1 = (((np.random.rand(img.shape[0]*img.shape[1]))>0.3).reshape(img.shape)/0.7).astype('float32')
  r =tf.convert_to_tensor (x *U1)
  logits = tf.matmul(r,w) + b
  return logits

In [0]:
def prediction(x):
  pred = tf.matmul(x,w) + b
  return pred

In [0]:
#evaluate the loss

variables=[w,b]
def loss(model, img, label):
  prediction = model(img)
  return tf.losses.softmax_cross_entropy(label, logits=prediction)

#record the gradient with respect to the model variables 
def grad(model, img, label):
  with tf.GradientTape() as tape:
    loss_value = loss(model, img, label)
  return tape.gradient(loss_value, variables)




  

In [0]:

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
n_batch=train.images.shape[0]//batch_size

In [46]:
for i in range(n_epochs):
  for j in range(n_batch):
    img , label=train.next_batch(batch_size= batch_size)
    img = pca.transform(img)
    grads=grad(model,img,label)
    optimizer.apply_gradients(zip(grads,variables),global_step=tf.train.get_or_create_global_step())
  #train acc
  input_data = pca.transform(train.images)
  label_value=train.labels
  preds = tf.nn.softmax(prediction(input_data))
  correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label_value, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
  print(f'training accuracy after {i} epochs is {accuracy}')
  #test acc
  input_data = pca.transform(test.images)
  label_value=test.labels
  preds = tf.nn.softmax(prediction(input_data))
  correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label_value, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
  print(f'test accuracy after {i+1} epochs is {accuracy}')


training accuracy after 0 epochs is 0.8080545663833618
test accuracy after 1 epochs is 0.8012999892234802
training accuracy after 1 epochs is 0.8270727396011353
test accuracy after 2 epochs is 0.8167999982833862
training accuracy after 2 epochs is 0.8355273008346558
test accuracy after 3 epochs is 0.8252999782562256
training accuracy after 3 epochs is 0.8396545648574829
test accuracy after 4 epochs is 0.8281999826431274
training accuracy after 4 epochs is 0.8434000015258789
test accuracy after 5 epochs is 0.8313999772071838
training accuracy after 5 epochs is 0.8444545269012451
test accuracy after 6 epochs is 0.8306999802589417
training accuracy after 6 epochs is 0.8471272587776184
test accuracy after 7 epochs is 0.8309000134468079
training accuracy after 7 epochs is 0.8483818173408508
test accuracy after 8 epochs is 0.8309999704360962
training accuracy after 8 epochs is 0.8493454456329346
test accuracy after 9 epochs is 0.83160001039505
training accuracy after 9 epochs is 0.8497818112